<a href="https://colab.research.google.com/github/merveenbiya/YL_TEZ/blob/main/Model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [34]:
pip install pandas scikit-learn xgboost


# SALT MODEL

In [45]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SelectFromModel
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [41]:
# Load the dataset
file_path = '/content/drive/MyDrive/Merve Enbiyaoğlu Tez/data/data_fiiligecis_ogle_ogrenci.csv'
data = pd.read_csv(file_path)

In [44]:
# Separate features (X) and target variable (y)
X = data.drop(['fiili_gecis_ogrenci_ogle'], axis=1)
y = data['fiili_gecis_ogrenci_ogle']

# Identify categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_columns]))
X_encoded.columns = encoder.get_feature_names_out(categorical_columns)

# Concatenate the one-hot encoded features with the remaining features
X = pd.concat([X.drop(categorical_columns, axis=1), X_encoded], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize XGBoost model
model = XGBRegressor()

# Fit the model on the training data
model.fit(X_train, y_train)

# Perform feature selection
feature_selector = SelectFromModel(model, threshold=-np.inf, prefit=True)
X_train_selected = feature_selector.transform(X_train)
X_test_selected = feature_selector.transform(X_test)

# Print selected features
selected_features = X.columns[feature_selector.get_support()]
print("Selected Features:", selected_features)

# Train XGBoost model on the selected features
model_selected = XGBRegressor()
model_selected.fit(X_train_selected, y_train)

# Make predictions on the test set
y_pred = model_selected.predict(X_test_selected)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Selected Features: Index(['ramazan_ayi', 'yilin_ayi', 'toplam_kalori', 'beklenen_hava_sicakligi',
       'anlik_hava_sicakligi', 'ogrenci_kyk_odenme_takvimi',
       'yillik_enflasyon_orani', 'aylik_enflasyon_orani',
       'corba_anadolu çorbası', 'corba_arabaşı çorbası',
       ...
       'beklenen_hava_olayi_gökgürültülü sağanak yağışlı',
       'beklenen_hava_olayi_hafif sağanak yağışlı',
       'beklenen_hava_olayi_hafif yağmurlu',
       'beklenen_hava_olayi_parçalı bulutlu',
       'beklenen_hava_olayi_sağanak yağışlı', 'beklenen_hava_olayi_yağmurlu',
       'anlik_hava_olayi_az bulutlu', 'anlik_hava_olayi_açık',
       'anlik_hava_olayi_parçalı bulutlu', 'anlik_hava_olayi_çok bulutlu'],
      dtype='object', length=120)
Mean Squared Error on Test Set: 71643.38246108136


# W GRIDSEARCH


In [46]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

In [47]:
file_path = '/content/drive/MyDrive/Merve Enbiyaoğlu Tez/data/data_fiiligecis_ogle_ogrenci.csv'
data = pd.read_csv(file_path)

In [48]:
# Separate features (X) and target variable (y)
X = data.drop(['fiili_gecis_ogrenci_ogle'], axis=1)
y = data['fiili_gecis_ogrenci_ogle']

# Identify categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns

# One-hot encode categorical columns
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_columns]))
X_encoded.columns = encoder.get_feature_names_out(categorical_columns)

# Concatenate the one-hot encoded features with the remaining features
X = pd.concat([X.drop(categorical_columns, axis=1), X_encoded], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the XGBoost model
model = XGBRegressor()

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5, verbose=2)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Get the best parameters from the grid search
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# Use the best model for predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.8; total time=   0.2s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.1s
[CV] END colsample_bytree=0.8, learning_rate=0.01, max_depth=3, n_estimators=50, subsample=0.9; total time=   0.1s
[CV] END colsamp